# Accelerate 실습

## Accelerate란?
**Hugging Face Accelerate**는 복잡한 분산 학습을 간단하게 만들어주는 도구입니다.

### 핵심 기능
- 기존 PyTorch 코드를 거의 그대로 사용
- 몇 줄만 추가하면 분산 학습 가능
- 다양한 하드웨어 환경을 자동으로 감지하고 최적화


## 기본 사용법

### 핵심 차이점
**기존 PyTorch:**
```python
model.to(device)
for batch in dataloader:
    batch = batch.to(device)
    loss.backward()
```

**Accelerate 사용:**
```python
from accelerate import Accelerator
accelerator = Accelerator()
model, optimizer, dataloader = accelerator.prepare(model, optimizer, dataloader)

for batch in dataloader:
    accelerator.backward(loss)  # 디바이스 이동 자동 처리
```


# 실습: NSMC 데이터셋으로 감정 분석 파인튜닝

- **모델**: KcBERT (한국어 특화 BERT)
- **작업**: 감정 분석 (긍정/부정)
- **데이터**: NSMC (네이버 영화 리뷰 감정 분석 데이터셋)


In [1]:
%pip install accelerate transformers datasets evaluate


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 라이브러리 import
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from datasets import Dataset
from accelerate import Accelerator
import evaluate
from tqdm.auto import tqdm
import pandas as pd
import urllib.request
import os

print("라이브러리 import 완료!")


라이브러리 import 완료!


In [3]:
# Accelerator 초기화
accelerator = Accelerator()
print(f"사용 디바이스: {accelerator.device}")
print("Accelerator 초기화 완료!")


사용 디바이스: cuda
Accelerator 초기화 완료!


In [4]:
# NSMC 데이터셋 다운로드 및 로드 (e9t/nsmc)
def download_nsmc_dataset():
    """NSMC 데이터셋을 다운로드하고 로드하는 함수"""
    
    base_url = "https://raw.githubusercontent.com/e9t/nsmc/master/"
    files = {
        "train": "ratings_train.txt",
        "test": "ratings_test.txt"
    }
    
    data = {}
    
    for split, filename in files.items():
        file_path = f"./{filename}"
        
        # 파일이 없으면 다운로드
        if not os.path.exists(file_path):
            print(f"{filename} 다운로드 중...")
            url = base_url + filename
            urllib.request.urlretrieve(url, file_path)
            print(f"✅ {filename} 다운로드 완료")
        
        # 데이터 로드 (TSV 파일, 탭으로 구분)
        df = pd.read_csv(file_path, sep='\t', encoding='utf-8')
        
        # NaN 값 제거
        df = df.dropna()
        
        # 데이터 개수 제한 (학습 속도를 위해)
        if split == "train":
            df = df.head(1000)  # 훈련 데이터 1000개
        else:
            df = df.head(200)   # 테스트 데이터 200개
        
        data[split] = df
        print(f"{split} 데이터 로드: {len(df)}개")
    
    return data

# NSMC 데이터셋 다운로드 및 로드
nsmc_data = download_nsmc_dataset()

# Dataset 객체로 변환
train_df = nsmc_data["train"]
test_df = nsmc_data["test"]

dataset = Dataset.from_dict({
    "document": train_df["document"].tolist(),
    "label": train_df["label"].tolist()
})

test_dataset = Dataset.from_dict({
    "document": test_df["document"].tolist(),
    "label": test_df["label"].tolist()
})

print(f"\n✅ NSMC 데이터셋 준비 완료!")
print(f"훈련 데이터: {len(dataset)}개")
print(f"테스트 데이터: {len(test_dataset)}개")
print(f"예시 리뷰: {dataset[0]['document'][:50]}...")
print(f"라벨: {dataset[0]['label']} (0=부정, 1=긍정)")


ratings_train.txt 다운로드 중...
✅ ratings_train.txt 다운로드 완료
train 데이터 로드: 1000개
ratings_test.txt 다운로드 중...
✅ ratings_test.txt 다운로드 완료
test 데이터 로드: 200개

✅ NSMC 데이터셋 준비 완료!
훈련 데이터: 1000개
테스트 데이터: 200개
예시 리뷰: 아 더빙.. 진짜 짜증나네요 목소리...
라벨: 0 (0=부정, 1=긍정)


In [5]:
# 한국어 특화 모델과 토크나이저 로드
model_name = "beomi/kcbert-base"  # 한국어 특화 BERT 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

print(f"✅ KcBERT 모델 로드 완료: {model_name}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ KcBERT 모델 로드 완료: beomi/kcbert-base


In [6]:
# 데이터 전처리 (모델의 최대 길이에 맞게 설정)
# 모델의 최대 시퀀스 길이 확인
max_length = min(tokenizer.model_max_length, 300)  # KcBERT는 보통 300이 안전
print(f"사용할 최대 길이: {max_length}")

def preprocess_function(examples):
    result = tokenizer(examples["document"], truncation=True, padding="max_length", max_length=max_length)
    result["labels"] = examples["label"]
    return result

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["document", "label"])
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=["document", "label"])

print(f"데이터 전처리 완료! (최대 길이: {max_length})")


사용할 최대 길이: 300


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

데이터 전처리 완료! (최대 길이: 300)


In [7]:
# 데이터로더 생성
tokenized_dataset.set_format("torch")
tokenized_test_dataset.set_format("torch")

batch_size = 4  # 메모리 안정성을 위해 배치 크기 감소
train_dataloader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(tokenized_test_dataset, batch_size=batch_size, shuffle=False)

print(f"데이터로더 생성 완료 (배치 크기: {batch_size})")


데이터로더 생성 완료 (배치 크기: 4)


In [8]:
# 학습 설정
learning_rate = 5e-5
num_epochs = 3
optimizer = AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_scheduler("linear", optimizer, 0, num_epochs * len(train_dataloader))
accuracy_metric = evaluate.load("accuracy")

print(f"학습 설정 완료 (에포크: {num_epochs}, 학습률: {learning_rate})")


학습 설정 완료 (에포크: 3, 학습률: 5e-05)


In [9]:
# Accelerate prepare (핵심!)
model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
)

print("Accelerate prepare 완료!")


Accelerate prepare 완료!


In [10]:
# 평가 함수
def evaluate_model():
    model.eval()
    all_predictions, all_labels = [], []
    
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)
            predictions = outputs.logits.argmax(dim=-1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(batch["labels"].cpu().numpy())
    
    accuracy = accuracy_metric.compute(predictions=all_predictions, references=all_labels)["accuracy"]
    model.train()
    return accuracy

initial_accuracy = evaluate_model()
print(f"초기 정확도: {initial_accuracy:.4f}")


초기 정확도: 0.5650


In [11]:
# 파인튜닝 학습
print("파인튜닝 시작!")
progress_bar = tqdm(range(num_epochs * len(train_dataloader)))

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    
    eval_accuracy = evaluate_model()
    avg_loss = total_loss / len(train_dataloader)
    
    print(f"Epoch {epoch + 1}: 손실={avg_loss:.4f}, 정확도={eval_accuracy:.4f}")

print(f"파인튜닝 완료! 성능 향상: {eval_accuracy - initial_accuracy:.4f}")


파인튜닝 시작!


  0%|          | 0/750 [00:00<?, ?it/s]

Epoch 1: 손실=0.5342, 정확도=0.7750
Epoch 2: 손실=0.2247, 정확도=0.8600
Epoch 3: 손실=0.0659, 정확도=0.8450
파인튜닝 완료! 성능 향상: 0.2800


In [15]:
import torch 

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [16]:
# 모델 저장 및 테스트
model_save_path = "./fine_tuned_klue_bert_sentiment"
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

# 예측 테스트
test_sentences = ["이 영화 정말 재미있어요!", "별로였어요. 시간 낭비"]
unwrapped_model.eval()

for text in test_sentences:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = unwrapped_model(**inputs)
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(prediction, dim=-1).item()
    
    sentiment = "긍정" if predicted_class == 1 else "부정"
    confidence = prediction[0][predicted_class].item()
    print(f"'{text}' → {sentiment} ({confidence:.3f})")

print("모델 저장 및 테스트 완료!")


'이 영화 정말 재미있어요!' → 긍정 (0.989)
'별로였어요. 시간 낭비' → 부정 (0.997)
모델 저장 및 테스트 완료!


## 핵심 포인트

### Accelerate 사용법
1. `accelerator = Accelerator()` - 초기화
2. `accelerator.prepare()` - 모델, 옵티마이저, 데이터로더 준비
3. `accelerator.backward(loss)` - 역전파

### 장점
- 기존 PyTorch 코드 재사용 가능
- 자동 분산 학습 지원
- 디바이스 관리 자동화
